In [1]:
import app_functions.data_manipulation as dm
import app_functions.requests as req

In [3]:
import pandas as pd
config = dm.read_config()

# read sport as sport field in config
sports = config["sports"]
token = config["api_token"]
bookmakers = config["bookmakers"]

#test
# get odds for each sport
all_odds = [] 
for sport in sports:
    odds = req.get_test_odds_request(f"{sport}_odds.json") #req.get_odds_request(token, s, bookmakers)
    all_odds.append(odds)


# convert to dataframe
df = dm.json_to_df(all_odds)

In [4]:
df.head()

,sport,teams,bookmaker,bet type,event date,favorite,underdog,odds favorite,odds underdog,game_id
0,NFL,Baltimore Ravens_Kansas City Chiefs,draftkings,h2h,2024-09-06,Kansas City Chiefs,Baltimore Ravens,-148,124,Baltimore Ravens_Kansas City Chiefs_2024-09-06...
1,NFL,Baltimore Ravens_Kansas City Chiefs,fanduel,h2h,2024-09-06,Kansas City Chiefs,Baltimore Ravens,-152,128,Baltimore Ravens_Kansas City Chiefs_2024-09-06...
2,NFL,Baltimore Ravens_Kansas City Chiefs,betmgm,h2h,2024-09-06,Kansas City Chiefs,Baltimore Ravens,-155,130,Baltimore Ravens_Kansas City Chiefs_2024-09-06...
3,NFL,Green Bay Packers_Philadelphia Eagles,draftkings,h2h,2024-09-07,Philadelphia Eagles,Green Bay Packers,-142,120,Green Bay Packers_Philadelphia Eagles_2024-09-...
4,NFL,Green Bay Packers_Philadelphia Eagles,fanduel,h2h,2024-09-07,Philadelphia Eagles,Green Bay Packers,-152,128,Green Bay Packers_Philadelphia Eagles_2024-09-...


In [5]:
import itertools

new_df = pd.DataFrame(columns=['sport','game_id', 'bookie1', 'bookie2'])

for game_id, group in df.groupby('game_id'):
    sport = group['sport'].iloc[0]
    bookies = group['bookmaker'].unique()
    for bookie1, bookie2 in itertools.combinations(bookies, 2):
        new_index = len(new_df)
        new_row = {'game_id': game_id, 'bookie1': bookie1, 'bookie2': bookie2, 'sport': sport}
        new_df.loc[new_index] = new_row

In [6]:
new_df.head()

,sport,game_id,bookie1,bookie2
0,NCAAF,Air Force Falcons_Merrimack Warriors_2024-08-3...,fanduel,draftkings
1,NCAAF,Alabama Crimson Tide_Western Kentucky Hilltopp...,draftkings,fanduel
2,NCAAF,Alabama Crimson Tide_Wisconsin Badgers_2024-09...,betmgm,fanduel
3,NFL,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,draftkings
4,NFL,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,betmgm


In [7]:
new_df.drop_duplicates(inplace=True)
new_df.dropna(inplace=True)
new_df.reset_index(drop=True, inplace=True)

In [8]:
def get_bookie_team(row, bookie_num, type):
    # Build the column name dynamically based on bookie number and type
    odds_column = "favorite" if type == "favorite" else "underdog"
    
    filtered_df = df[(df["bookmaker"] == row[bookie_num]) & (df["game_id"] == row["game_id"])]

    # Assuming there's only one matching row, get the specified odds value
    if not filtered_df.empty:
        return filtered_df[odds_column].iloc[0]
    else:
        return None  # or some default value
    
def get_odds(row, bookie_num, type):
    odds_column = "odds favorite" if type == "favorite" else "odds underdog"
    
    # Filter df for the current row's bookie and game_id
    filtered_df = df[(df["bookmaker"] == row[bookie_num]) & (df["game_id"] == row["game_id"])]
    
    # Assuming there's only one matching row, get the specified odds value
    if not filtered_df.empty:
        return filtered_df[odds_column].iloc[0]
    else:
        return None  # or some default value

In [9]:
new_df["bookie1_favorite_odds"] = new_df.apply(get_odds, bookie_num='bookie1', type='favorite', axis=1)
new_df["bookie1_favorite"] = new_df.apply(get_bookie_team, bookie_num = 'bookie1', type='favorite', axis=1)
new_df["bookie2_favorite_odds"] = new_df.apply(get_odds, bookie_num='bookie2', type='favorite', axis=1)
new_df["bookie2_favorite"] = new_df.apply(get_bookie_team, bookie_num = 'bookie2', type='favorite', axis=1)
new_df["bookie1_underdog_odds"] = new_df.apply(get_odds, bookie_num='bookie1', type='underdog', axis=1)
new_df["bookie1_underdog"] = new_df.apply(get_bookie_team, bookie_num = 'bookie1', type='underdog', axis=1)
new_df["bookie2_underdog_odds"] = new_df.apply(get_odds, bookie_num='bookie2', type='underdog', axis=1)
new_df["bookie2_underdog"] = new_df.apply(get_bookie_team, bookie_num = 'bookie2', type='underdog', axis=1)

In [10]:
new_df.head()

,sport,game_id,bookie1,bookie2,bookie1_favorite_odds,bookie1_favorite,bookie2_favorite_odds,bookie2_favorite,bookie1_underdog_odds,bookie1_underdog,bookie2_underdog_odds,bookie2_underdog
0,NCAAF,Air Force Falcons_Merrimack Warriors_2024-08-3...,fanduel,draftkings,-20000,Air Force Falcons,-6500,Air Force Falcons,3500,Merrimack Warriors,2000,Merrimack Warriors
1,NCAAF,Alabama Crimson Tide_Western Kentucky Hilltopp...,draftkings,fanduel,-10000,Alabama Crimson Tide,-10000,Alabama Crimson Tide,3000,Western Kentucky Hilltoppers,3000,Western Kentucky Hilltoppers
2,NCAAF,Alabama Crimson Tide_Wisconsin Badgers_2024-09...,betmgm,fanduel,-350,Alabama Crimson Tide,-330,Alabama Crimson Tide,270,Wisconsin Badgers,260,Wisconsin Badgers
3,NFL,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,draftkings,-260,Buffalo Bills,-278,Buffalo Bills,215,Arizona Cardinals,225,Arizona Cardinals
4,NFL,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,betmgm,-260,Buffalo Bills,-275,Buffalo Bills,215,Arizona Cardinals,225,Arizona Cardinals


In [11]:
new_df = new_df.dropna()

In [12]:
new_df["max_favorite_odds"] = new_df[["bookie1_favorite_odds", "bookie2_favorite_odds"]].max(axis=1)
new_df["max_underdog_odds"] = new_df[["bookie1_underdog_odds", "bookie2_underdog_odds"]].max(axis=1)
new_df["favorite_diff"] = (new_df["bookie1_favorite_odds"] - new_df["bookie2_favorite_odds"]).abs()

In [13]:
# create new df with only the rows with the max difference in odds for each game_id
diff_df = new_df.loc[new_df.groupby('game_id')['favorite_diff'].idxmax()]
diff_df.dropna(inplace=True)
diff_df.reset_index(drop=True, inplace=True)

In [14]:
diff_df.head()

,sport,game_id,bookie1,bookie2,bookie1_favorite_odds,bookie1_favorite,bookie2_favorite_odds,bookie2_favorite,bookie1_underdog_odds,bookie1_underdog,bookie2_underdog_odds,bookie2_underdog,max_favorite_odds,max_underdog_odds,favorite_diff
0,NCAAF,Air Force Falcons_Merrimack Warriors_2024-08-3...,fanduel,draftkings,-20000,Air Force Falcons,-6500,Air Force Falcons,3500,Merrimack Warriors,2000,Merrimack Warriors,-6500,3500,13500
1,NCAAF,Alabama Crimson Tide_Western Kentucky Hilltopp...,draftkings,fanduel,-10000,Alabama Crimson Tide,-10000,Alabama Crimson Tide,3000,Western Kentucky Hilltoppers,3000,Western Kentucky Hilltoppers,-10000,3000,0
2,NCAAF,Alabama Crimson Tide_Wisconsin Badgers_2024-09...,betmgm,fanduel,-350,Alabama Crimson Tide,-330,Alabama Crimson Tide,270,Wisconsin Badgers,260,Wisconsin Badgers,-330,270,20
3,NFL,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,draftkings,-260,Buffalo Bills,-278,Buffalo Bills,215,Arizona Cardinals,225,Arizona Cardinals,-260,225,18
4,NCAAF,Arizona State Sun Devils_Wyoming Cowboys_2024-...,fanduel,betmgm,-260,Arizona State Sun Devils,-275,Arizona State Sun Devils,210,Wyoming Cowboys,225,Wyoming Cowboys,-260,225,15


In [15]:
diff_df["favorite_bookie"] = diff_df.apply(lambda row: row["bookie1"] if row["bookie1_favorite_odds"] > row["bookie2_favorite_odds"] else row["bookie2"], axis=1)
diff_df["underdog_bookie"] = diff_df.apply(lambda row: row["bookie1"] if row["bookie1_underdog_odds"] > row["bookie2_underdog_odds"] else row["bookie2"], axis=1)

diff_df["favorite_bookie_team"] = diff_df.apply(lambda row: row["bookie1_favorite"] if row["bookie1_favorite_odds"] > row["bookie2_favorite_odds"] else row["bookie2_favorite"], axis=1)
diff_df["underdog_bookie_team"] = diff_df.apply(lambda row: row["bookie1_underdog"] if row["bookie1_underdog_odds"] > row["bookie2_underdog_odds"] else row["bookie2_underdog"], axis=1)

In [16]:
diff_df.head()

,sport,game_id,bookie1,bookie2,bookie1_favorite_odds,bookie1_favorite,bookie2_favorite_odds,bookie2_favorite,bookie1_underdog_odds,bookie1_underdog,bookie2_underdog_odds,bookie2_underdog,max_favorite_odds,max_underdog_odds,favorite_diff,favorite_bookie,underdog_bookie,favorite_bookie_team,underdog_bookie_team
0,NCAAF,Air Force Falcons_Merrimack Warriors_2024-08-3...,fanduel,draftkings,-20000,Air Force Falcons,-6500,Air Force Falcons,3500,Merrimack Warriors,2000,Merrimack Warriors,-6500,3500,13500,draftkings,fanduel,Air Force Falcons,Merrimack Warriors
1,NCAAF,Alabama Crimson Tide_Western Kentucky Hilltopp...,draftkings,fanduel,-10000,Alabama Crimson Tide,-10000,Alabama Crimson Tide,3000,Western Kentucky Hilltoppers,3000,Western Kentucky Hilltoppers,-10000,3000,0,fanduel,fanduel,Alabama Crimson Tide,Western Kentucky Hilltoppers
2,NCAAF,Alabama Crimson Tide_Wisconsin Badgers_2024-09...,betmgm,fanduel,-350,Alabama Crimson Tide,-330,Alabama Crimson Tide,270,Wisconsin Badgers,260,Wisconsin Badgers,-330,270,20,fanduel,betmgm,Alabama Crimson Tide,Wisconsin Badgers
3,NFL,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,draftkings,-260,Buffalo Bills,-278,Buffalo Bills,215,Arizona Cardinals,225,Arizona Cardinals,-260,225,18,fanduel,draftkings,Buffalo Bills,Arizona Cardinals
4,NCAAF,Arizona State Sun Devils_Wyoming Cowboys_2024-...,fanduel,betmgm,-260,Arizona State Sun Devils,-275,Arizona State Sun Devils,210,Wyoming Cowboys,225,Wyoming Cowboys,-260,225,15,fanduel,betmgm,Arizona State Sun Devils,Wyoming Cowboys


In [17]:
needed_cols = ["sport","game_id", "favorite_bookie", "favorite_bookie_team", "max_favorite_odds", "underdog_bookie", "underdog_bookie_team", "max_underdog_odds"]
arbitrage_df = diff_df[needed_cols]
arbitrage_df["arb_available"] = False

C:\Users\owurs\AppData\Local\Temp\ipykernel_1892\3787717607.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arbitrage_df["arb_available"] = False


In [18]:
arbitrage_df.head()

,sport,game_id,favorite_bookie,favorite_bookie_team,max_favorite_odds,underdog_bookie,underdog_bookie_team,max_underdog_odds,arb_available
0,NCAAF,Air Force Falcons_Merrimack Warriors_2024-08-3...,draftkings,Air Force Falcons,-6500,fanduel,Merrimack Warriors,3500,False
1,NCAAF,Alabama Crimson Tide_Western Kentucky Hilltopp...,fanduel,Alabama Crimson Tide,-10000,fanduel,Western Kentucky Hilltoppers,3000,False
2,NCAAF,Alabama Crimson Tide_Wisconsin Badgers_2024-09...,fanduel,Alabama Crimson Tide,-330,betmgm,Wisconsin Badgers,270,False
3,NFL,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,Buffalo Bills,-260,draftkings,Arizona Cardinals,225,False
4,NCAAF,Arizona State Sun Devils_Wyoming Cowboys_2024-...,fanduel,Arizona State Sun Devils,-260,betmgm,Wyoming Cowboys,225,False


In [19]:
# print rows where bookies have different favorite teams
# prints the obvious arbitrage opportunities (there are rarely any)
for index, row in diff_df.iterrows():
    if (row["bookie1_favorite"] != row["bookie2_favorite"] and (row["bookie1_favorite_odds"] > 100) and (row["bookie2_favorite_odds"] > 100)):
        print(row)

In [20]:
# add bet amount columns for when arb is available and arb estimated profit
arbitrage_df["favorite_bet_amount"] = 0
arbitrage_df["underdog_bet_amount"] = 0
arbitrage_df["arb_min_profit"] = 0

C:\Users\owurs\AppData\Local\Temp\ipykernel_1892\3080679581.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arbitrage_df["favorite_bet_amount"] = 0
C:\Users\owurs\AppData\Local\Temp\ipykernel_1892\3080679581.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arbitrage_df["underdog_bet_amount"] = 0
C:\Users\owurs\AppData\Local\Temp\ipykernel_1892\3080679581.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [21]:
def fav_bet_winnings(bet_amount, odds):
    return ((bet_amount * 100 / -(odds)) + bet_amount)

def dog_bet_winnings(bet_amount, odds):
    return ((bet_amount * odds / 100) + bet_amount)

def bet_winnings(bet_amount, odds):
    if odds > 100:
        return fav_bet_winnings(bet_amount, odds)
    else:
        return dog_bet_winnings(bet_amount, odds)
    

In [22]:
def arb_available(dog_odds, dog_bet, fav_odds, fav_bet):
    if((bet_winnings(dog_bet, dog_odds) > dog_bet + fav_bet) and (bet_winnings(fav_bet, fav_odds) > dog_bet + fav_bet)):
        return True
        
    return False

In [23]:
def set_arb(row):
    dog_odds = row["max_underdog_odds"]
    fav_odds = row["max_favorite_odds"]

    if (fav_odds > 100 and dog_odds > 100):
        row["arb_available"] = True
        row["favorite_bet_amount"] = 50
        row["underdog_bet_amount"] = 50
        row["arb_min_profit"] = 0

    min_distance = 100000
    optimal_i = 0
    # cant lose more than the full combined bet
    min_profit = -100
    can_arb = False

    for i in range(1, 99):
        fav_bet = i
        dog_bet = 100 - i

        dog_winnings = dog_bet_winnings(dog_bet, dog_odds)
        fav_winnings = fav_bet_winnings(fav_bet, fav_odds)

        profit = min(dog_winnings, fav_winnings) - 100

        if(arb_available(dog_odds, dog_bet, fav_odds, fav_bet)):
            can_arb = True

        diff = abs(dog_winnings - fav_winnings)
        if diff < min_distance:
            min_distance = diff
            optimal_i = i
            min_profit = profit
    
    row["arb_available"] = can_arb
    row["favorite_bet_amount"] = optimal_i
    row["underdog_bet_amount"] = 100 - optimal_i
    row["arb_min_profit"] = min_profit

    return row

In [24]:
# apply the set_arb function to each row in the arbitrage_df
arbitrage_df = arbitrage_df.apply(set_arb, axis=1)

In [25]:
arbitrage_df.head()

,sport,game_id,favorite_bookie,favorite_bookie_team,max_favorite_odds,underdog_bookie,underdog_bookie_team,max_underdog_odds,arb_available,favorite_bet_amount,underdog_bet_amount,arb_min_profit
0,NCAAF,Air Force Falcons_Merrimack Warriors_2024-08-3...,draftkings,Air Force Falcons,-6500,fanduel,Merrimack Warriors,3500,False,97,3,-1.507692
1,NCAAF,Alabama Crimson Tide_Western Kentucky Hilltopp...,fanduel,Alabama Crimson Tide,-10000,fanduel,Western Kentucky Hilltoppers,3000,False,97,3,-7.000000
2,NCAAF,Alabama Crimson Tide_Wisconsin Badgers_2024-09...,fanduel,Alabama Crimson Tide,-330,betmgm,Wisconsin Badgers,270,False,74,26,-3.800000
3,NFL,Arizona Cardinals_Buffalo Bills_2024-09-08_h2h,fanduel,Buffalo Bills,-260,draftkings,Arizona Cardinals,225,False,70,30,-3.076923
4,NCAAF,Arizona State Sun Devils_Wyoming Cowboys_2024-...,fanduel,Arizona State Sun Devils,-260,betmgm,Wyoming Cowboys,225,False,70,30,-3.076923


In [26]:
def get_valid_arbs(arbitrage_df):
    return arbitrage_df[arbitrage_df["arb_available"] == True]

In [27]:
valid_arbs_df = get_valid_arbs(arbitrage_df)

In [28]:
def display_arbs(all_valid_arbs):
    # loop through all valid arbs and display them
    for index, row in all_valid_arbs.iterrows():
        print("Sport: " + row["sport"] + " Game ID: " + row["game_id"])
        print("\tBet " + str(row["favorite_bet_amount"]) + " on " + row["favorite_bookie_team"] + " at " + str(row["max_favorite_odds"]) + " on " + row["favorite_bookie"])
        print("\tBet " + str(row["underdog_bet_amount"]) + " on " + row["underdog_bookie_team"] + " at " + str(row["max_underdog_odds"]) + " on " + row["underdog_bookie"])
        
        print("\tArbitrage Attempt Minimum Profit: $" + str(row["arb_min_profit"]))
        print("\n")

In [29]:
# create fake df with all the columns
fake_arb_df = pd.DataFrame(columns=['sport','game_id', 'favorite_bookie', 'favorite_bookie_team', 'max_favorite_odds', 'underdog_bookie', 'underdog_bookie_team', 'max_underdog_odds', 'arb_available', 'favorite_bet_amount', 'underdog_bet_amount', 'arb_min_profit'])
new_row = {'sport': 'Basketball', 'game_id': '123', 'favorite_bookie': 'bookie1', 'favorite_bookie_team': 'team1', 'max_favorite_odds': 110, 'underdog_bookie': 'bookie2', 'underdog_bookie_team': 'team2', 'max_underdog_odds': 110, 'arb_available': True, 'favorite_bet_amount': 50, 'underdog_bet_amount': 50, 'arb_min_profit': 3.20}

# Get the current length of the DataFrame
current_length = len(fake_arb_df)

# Use loc to assign the new row at the next available index
fake_arb_df.loc[current_length] = new_row
display_arbs(fake_arb_df)

Sport: Basketball Game ID: 123
	Bet 50 on team1 at 110 on bookie1
	Bet 50 on team2 at 110 on bookie2
	Arbitrage Attempt Minimum Profit: $3.2




In [30]:
display_arbs(valid_arbs_df)

In [31]:
valid_arbs_df

,sport,game_id,favorite_bookie,favorite_bookie_team,max_favorite_odds,underdog_bookie,underdog_bookie_team,max_underdog_odds,arb_available,favorite_bet_amount,underdog_bet_amount,arb_min_profit
